In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
from pyhank import HankelTransform as pyhankHT

In [2]:
# We need to import spline interpolation for the inverse transformation to get smoothness
from scipy.interpolate import InterpolatedUnivariateSpline as spline

## Chen and Oldenburg 2004 implementation of the 2 layer system

In [3]:
# Electrode is placed at the interface between rho1 and rho2
class halfspace_2_layer:
    def __init__(self, rho1, rho2, h_overburden, tx_current):
        """
        Returns the field magnitude of a buried electrode located at the interface between media
        rho1 and rho2, thickness of rho1 being defined with h_overburden
        @return: B_field magnitude
        """
        
        self.rho1 = rho1
        self.rho2 = rho2
        self.h_overburden = h_overburden
        
        self.tx_current = tx_current
        
    
    def normal_B_Field(self, r, depth):
        transformer = pyhankHT(order=1, radial_grid=r)
        lbda = transformer.kr
        HankField = np.append(self.lower_layer_bfield(lbda, 
                                depth[depth <= self.h_overburden]),
                           self.upper_layer_bfield(lbda, 
                                depth[depth > self.h_overburden]), 
                           axis=0)
        BField = transformer.iqdht(HankField)
        return BField
    
    def upper_layer_bfield(self, lbda, depth):
        # lbda: float
        # depth: float
        # returns: ndarray of shape(47, len(depth))
        # Equation 15 from Chen and Oldenburg (2004)
        D1 = self._D_1(lbda)
        U1 = self._U_1(lbda)
        
        ## Shape of (n_points, 47)
        term1 = self.tx_current / 2 / np.pi / lbda
        term2 = D1 * np.exp(-lbda * depth) 
        term3 = U1 * np.exp(lbda * depth)
        return term1 + term2 + term3
    
    def lower_layer_bfield(self, lbda, depth):
        # lbda: ndarray
        # depth: ndarray
        # Equation 16 from Chen and Oldenburg (2004)
        D2 = self._D_2(lbda)
        U2 = self._U_2(lbda)
        print(lbda)
        term1 = D2 * np.exp(-lbda * depth[:,np.newaxis])
        term2 = U2 * np.exp(lbda * depth[:,np.newaxis])
        return term1 + term2
    
    ### LAYER COEFFICIENTS ###
    def _D_1(self, lbda):
        # lbda: ndarray of len(gupFilter)
        # Equation 17 from Chen and Oldenburg (2004)
        h1 = self.h_overburden
        
        bot = (1-np.exp(-2*lbda*h1)) + self.rho2 / self.rho1 * (1+np.exp(-2*lbda*h1))
        return self.tx_current * np.exp(-lbda * h1) / 2 / np.pi / lbda / bot
    
    def _U_1(self, lbda):
        # lbda: ndarray
        return -self._D_1(lbda)
    
    def _D_2(self, lbda):
        # lbda: ndndarray of len(gupFilter)array
        # Equation 18 from Chen and Oldenburg (2004)
        h1 = self.h_overburden
        
        top = (1 + np.exp(-2 * lbda * h1))
        bot = (1 - np.exp(-2 * lbda * h1)) + self.rho2 / self.rho1 * (1 + np.exp(-2*lbda*h1))
        return self.tx_current * top * self.rho2 / self.rho1 / 2 / np.pi / lbda / bot
    
    def _U_2(self, lbda):
        return 0


In [4]:
# Initialize a
earth = halfspace_2_layer(
                  rho1=100,
                  rho2=50,
                  h_overburden=50,
                  tx_current=1)

n=47 Filter loaded
[ 3.17926147e-06 -9.73811661e-06  1.64866227e-05 -1.81501261e-05
  1.87556556e-05 -1.14655041e-06  1.53799734e-05 -6.95628274e-06
  1.41881556e-05  3.41445666e-06  2.13941716e-05  2.34962369e-05
  4.84340283e-05  7.33732979e-05  1.27703784e-04  2.08120026e-04
  3.49803899e-04  5.79107815e-04  9.65887918e-04  1.60401274e-03
  2.66903778e-03  4.43111590e-03  7.35631696e-03  1.21782796e-02
  2.01097829e-02  3.30096953e-02  5.37143592e-02  8.60516613e-02
  1.34267607e-01  2.00125033e-01  2.74027506e-01  3.18168749e-01
  2.41655667e-01 -5.40549162e-02 -4.46912952e-01 -1.92231886e-01
  5.52376754e-01 -3.57429049e-01  1.41510519e-01 -4.61421935e-02
  1.48273762e-02 -5.07479209e-03  1.83829714e-03 -6.67742804e-04
  2.21277518e-04 -5.66248733e-05  7.88229203e-06]


[3.83170597]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 499 is different from 1)

In [ ]:
z